In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段6：开始使用Vertex AI批量预测为AutoML文本模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_text_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_text_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_text_model_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概述

本教程演示了如何从 `AutoML` 文本模型进行批量预测。

### 目标

在本教程中，您将学习如何使用`Vertex AI Batch Prediction`与一个`AutoML`文本模型。

本教程使用以下谷歌云机器学习服务和资源：

- `Vertex AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Models`

所执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练一个`AutoML`模型。
- 使用JSONL输入进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，后台运行，并在准备就绪时将结果存储在Cloud Storage存储桶中。

数据集

本教程使用的数据集是来自Kaggle数据集的[幸福时刻数据集](https://www.kaggle.com/ritresearch/happydb)。您在本教程中使用的数据集版本存储在公共云存储桶中。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[云存储
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您预计的使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南中提供了关于[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)的详细说明来满足这些要求。以下步骤提供了一组简洁的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow $USER_FLAG -q

重启内核

一旦您安装了额外的包，您需要重新启动笔记本的内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300 的免费信用，可用于支付计算和存储成本。

2. [确保您的项目已启用计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI APIs、Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中执行所有命令时使用正确的项目。

**注意**: Jupyter 会将以 `!` 为前缀的行视为 shell 命令，并会内插以 `$` 为前缀的 Python 变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 `REGION` 变量，该变量用于笔记本的其余部分操作。以下是 Vertex AI 支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行 Vertex AI 训练。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，可能正在使用共享测试账户或项目。为了避免资源创建时用户之间发生名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到本教程中创建的资源名称。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench Notebooks**，您的环境已经通过身份验证。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，然后选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，然后选择**存储对象管理员**。

点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

将服务帐号密钥的路径输入到下面的单元格中作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必须的。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在不同会话间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在你准备好开始创建你自己的AutoML文本分类模型了。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为在云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计数示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="Happy Moments" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### 创建并运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流水线，2）运行这个流水线。

#### 创建训练流水线

使用`AutoMLTextTrainingJob`类创建一个AutoML训练流水线，需要以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体抽取模型。
- `multi_label`：如果是分类任务，是单标签（False）还是多标签（True）。
- `sentiment_max`：如果是情感分析任务，情感值的最大值。

实例化的对象是训练流水线的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="happydb_" + UUID, prediction_type="classification"
)

print(dag)

#### 运行训练管道

接下来，通过调用`run`方法运行DAG来开始训练作业，参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

当`run`方法完成时，将返回`Model`资源。

训练管道的执行将持续最多4小时。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 批处理预测简介

批处理预测提供了离线批量处理大量预测请求的能力。资源仅在批处理过程中预留，并且在批处理请求完成后取消预留。结果存储在云存储中，与在线预测不同，在在线预测中结果作为HTTP响应数据包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的Web服务器必须支持JSONL格式，该Web服务器将其转换为模型输入界面或服务函数界面直接支持的格式。对于批处理预测，这个JSONL格式被称为`pivot`格式。

### 批处理预测作业的输入格式

批处理服务器接受以下AutoML文本模型的输入格式：

- JSONL

### 批处理预测作业的输出格式

批处理服务器接受以下AutoML文本模型的输出格式：

- JSONL


### 枢轴格式

批处理服务器将输入格式转换为`pivot`（JSONL）格式，如下所示：

**JSONL**

每个输入行（请求）应包含一个且仅一个有效的json值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器生成具有相同格式的枢轴数据。然后将生成的枢轴数据包装成一个有效载荷请求：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的CSV标题将始终被忽略。必须明确将字符串字段用双引号括起来，否则该行将被丢弃，并且会输出解析错误消息到错误文件。非引号的值总是以浮点数传输。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器会将每个输入行（请求）转换为JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每行都会被转换为JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器生成具有相同格式的枢轴数据。然后将生成的枢轴数据包装成一个有效载荷请求：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例由apache_beam.io.tfrecordio模块作为二进制读取。然后将二进制对象序列化为ASCII字符串。预测服务器负责知道恢复实例的解码器。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**文件列表**

文件列表格式包含文件列表。在“FileList”文件中的每一行都指定一个单独的文件路径，指定为云存储位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器读取文件为二进制。然后将二进制对象序列化为ASCII字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

### 使用JSONL输入创建批量输入文件

现在创建一个批量输入文件，将其存储在您的本地云存储桶中。在这个例子中，您可以使用JSONL格式作为输入和输出。对于输入，JSONL文件的布局如下：

    {'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

### 获取测试项

现在对您的 Vertex AI 模型进行批量预测。您将使用数据集中的任意示例作为测试项。不必担心这些示例很可能已经在训练模型时使用过 — 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2
if len(test_items[0]) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 创建批量输入文件

现在创建一个批量输入文件，您将存储在本地云存储存储桶中。批量输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）在每一行上创建一个字典条目。该字典包含键/值对：

- `content`：文本项文件的云存储路径。
- `mime_type`：内容类型。在我们的例子中，它是一个`text`文件。

例如：

{'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with open("test1.txt", "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with open("test2.txt", "w") as f:
    f.write(test_item_2 + "\n")

! gsutil cp test1.txt $gcs_test_item_1
! gsutil cp test2.txt $gcs_test_item_2

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with open("test.jsonl", "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cp test.jsonl $gcs_input_uri
! gsutil cat $gcs_input_uri

### 进行批处理预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法进行批处理预测，使用以下参数：

- `job_display_name`: 批处理预测任务的可读名称。
- `instances_format`: 批处理预测请求文件的格式： "jsonl", "csv", "bigquery", "tf-record", "tf-record-gzip" 或 "file-list"
- `prediction_format`: 批处理预测响应文件的格式： "jsonl", "csv", "bigquery", "tf-record", "tf-record-gzip" 或 "file-list"
- `gcs_source`: 一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`: 存储批处理预测结果的云存储位置。
- `sync`: 如果设置为True，则调用将阻塞，等待异步批处理任务完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="happydb_" + UUID,
    instances_format="jsonl",
    predictions_format="jsonl",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果被写入您在批量预测请求中指定的 Cloud Storage 输出存储桶中。您可以调用 `iter_outputs()` 方法来获取生成的与结果相关的每个 Cloud Storage 文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于 0 和 1 之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_batch_job = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)
if delete_batch_job:
    batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}